In [ ]:
import keras
import theano
import numpy as np
from itertools import islice, izip

import sys
sys.path.append('..')
from data_iters.archive.iam_iterator import IAM_MiniBatcher
from data_iters.archive.minibatcher import MiniBatcher

In [ ]:
from keras.layers import Dense, Flatten, Convolution2D, Activation
from keras.models import Graph, Sequential
from keras.objectives import mean_squared_error, binary_crossentropy, categorical_crossentropy
from keras.optimizers import SGD

from keras.utils.np_utils import to_categorical

In [ ]:
%pylab inline

In [ ]:
hdf5authors="/fileserver/iam/iam-processed/writing/authors_writing.hdf5"
num_authors=100
num_forms_per_author=5
shingle_dim=(56,56)
batch_size=12000
nb_epochs=10000
mini_m = IAM_MiniBatcher(hdf5authors, num_authors, num_forms_per_author,
                           shingle_dim=shingle_dim, default_mode=MiniBatcher.TRAIN,
                           batch_size=batch_size,train_pct=0.5,test_pct=0.25,val_pct=0.25)


In [ ]:
def gen_data(img_shape):
    canvas = np.zeros(img_shape)
    # choose label
    label = np.random.randint(0, 2)
    if label == 0:
        for i in canvas.shape[0]:
            for j in canvas.shape[1]:
                canvas = i + j
    
def make_graph(input_name, img_shape):
    graph = Graph()
    graph.add_input(name=input_name, input_shape=img_shape)
    return graph

def autoencoder_layers(graph, input_name, img_shape):
    graph.add_node(Flatten(), name='dnae_flatten1', input=input_name)
    graph.add_node(Dense(2000, activation='relu'), name='dnae_dense2', input='dnae_flatten1')
    graph.add_node(Dense(750, activation='relu'), name='dnae_dense3',  input='dnae_dense2')
    graph.add_node(Dense(1500, activation='relu'), name='dnae_dense4', input='dnae_dense3')
    graph.add_node(Dense(2500, activation='relu'), name='dnae_dense5', input='dnae_dense4')
    graph.add_node(Dense(np.prod(img_shape), activation='sigmoid'), name='dnae_reconstruction4', input='dnae_dense5')
    return 'dnae_reconstruction4'

def classifier_layers(graph, input_name, output_size):
    graph.add_node(Flatten(), 'cls_flatten1', input=input_name)
    graph.add_node(Dense(2000, activation='relu'), name='cls_dense2', input='cls_flatten1')
    graph.add_node(Dense(output_size, activation='sigmoid'), name='cls_dense3', input='cls_dense2')
    return 'cls_dense3'

scale_coef=2.0
def recon_loss(*args):
    return scale_coef * mean_squared_error(*args)

def tie_a_bow(graph, rc_layer_name, cl_layer_name):
    graph.add_output(name='rc_output', input=rc_layer_name)
    graph.add_output(name='cl_output', input=cl_layer_name)
    losses = { 'rc_output': recon_loss, 'cl_output': categorical_crossentropy }
    optimizer = SGD(lr=0.001, decay = 1e-6, momentum=0.9)
    graph.compile(optimizer=optimizer, loss=losses)
    return ['rc_output', 'cl_output']
    
def do_fitting(graph, batcher, nb_epochs, input_name, rc_output_name, cl_output_name, output_size):
    model_input = graph.get_input(train=False)['img_input']
    model_output = graph.nodes['dnae_reconstruction4'].get_output(train=False)
    getrecon = theano.function([model_input], model_output, allow_input_downcast=True)

    for i in range(nb_epochs):
        (X_train,Y_train) = batcher.get_train_batch()
        X_train = 1. - (X_train/255.)
        #X_train = np.expand_dims(X_train, 1)
        X_rec = X_train.reshape((X_train.shape[0], -1))[:]
        Y_train = to_categorical(Y_train, output_size)
        data_hash = {input_name: X_train, 
                     rc_output_name: X_rec,
                     cl_output_name: Y_train}
        recons = getrecon(X_train[:5])
        plt.figure()
        for j,(img,recon) in enumerate(zip(X_train[:5],recons)):
            plt.subplot(2,5,j+1)
            plt.imshow(img.reshape(X_train.shape[1],-1), cmap='gray')
            plt.subplot(2,5,5+j+1)
            plt.imshow(recon.reshape(X_train.shape[1],-1), cmap='gray')
        plt.show()
        history = graph.fit(data=data_hash, batch_size=X_train.shape[0], nb_epoch=1, verbose=2, validation_split=0.05)


In [ ]:

input_name='img_input'
graph = make_graph(input_name, shingle_dim)
rc_layer_name = autoencoder_layers(graph, input_name, shingle_dim)
cl_layer_name = classifier_layers(graph, input_name, num_authors)
rc_output_name, cl_output_name = tie_a_bow(graph, rc_layer_name, cl_layer_name)

In [ ]:
do_fitting(graph, mini_m, nb_epochs, input_name, rc_output_name, cl_output_name, num_authors)